In [1]:
# === Median ensemble for two submissions (Colab-ready) ===
import os
import numpy as np
import pandas as pd

# ---- 입력/출력 경로 설정 ----
PATH_A = "Pipeline_1_we_PEsemble_cs3.csv"
PATH_B = "submission_roomroom_hwadam추가.csv"
OUT_PATH = "JW_Tweedie_median.csv"   # 저장 파일명

# 경로가 없으면 현재 폴더의 동일 파일명으로 fallback
if not os.path.exists(PATH_A):
    alt = os.path.basename(PATH_A)
    if os.path.exists(alt):
        PATH_A = alt
if not os.path.exists(PATH_B):
    alt = os.path.basename(PATH_B)
    if os.path.exists(alt):
        PATH_B = alt

def read_csv_robust(p):
    try:
        return pd.read_csv(p)
    except UnicodeDecodeError:
        return pd.read_csv(p, encoding="cp949")

# ---- 로드 ----
df1 = read_csv_robust(PATH_A)  # 기준 스키마
df2 = read_csv_robust(PATH_B)

# ---- 정렬/정합 (ID 기준 병합으로 행 순서 보존) ----
id_col = df1.columns[0]
assert id_col in df2.columns, f"두 파일에 공통 ID열('{id_col}')이 필요합니다."

# 예측 컬럼 공통부분만 사용(순서는 df1 기준 유지)
pred_cols = [c for c in df1.columns[1:] if c in df2.columns]

# df2를 df1의 ID 순서에 맞추기
m1 = df1[[id_col] + pred_cols].copy()
m2 = df2[[id_col] + pred_cols].copy().set_index(id_col).reindex(m1[id_col]).reset_index()

# ---- 메디안 앙상블 → 반올림 → 최소 1로 클립 ----
A = m1[pred_cols].to_numpy(dtype=float)
B = m2[pred_cols].to_numpy(dtype=float)

# 음수 방지(선택적)
A = np.maximum(A, 0.0)
B = np.maximum(B, 0.0)

# 메디안
med = np.median(np.stack([A, B], axis=0), axis=0)

# 반올림(기본 rint). 파이프라인 임계 반올림을 쓰려면 아래 주석을 해제
ens = np.rint(med)
# BEST_THR = 0.130
# frac = med - np.floor(med)
# ens = np.where(frac >= BEST_THR, np.floor(med)+1, np.floor(med))

ens = np.clip(ens, 1, None).astype(np.int64)  # 최소 1

# ---- 결과 프레임 구성(스키마는 df1 기준 유지) ----
out = m1[[id_col]].copy()
out[pred_cols] = ens

# df1에만 있고 df2에는 없는 열은 df1 값 사용
extra_cols = [c for c in df1.columns[1:] if c not in pred_cols]
if extra_cols:
    out[extra_cols] = df1[extra_cols].values

# df1 열 순서 그대로 재정렬
out = out[df1.columns]

# ---- 저장 ----
out.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")

# ---- 리포트 ----
changed = None
try:
    base_vals = df1[pred_cols].to_numpy(dtype=float)
    changed = np.mean((np.rint(base_vals) != ens))
except Exception:
    pass

print(f"[DONE] Saved -> {OUT_PATH}")
print(f"Rows: {out.shape[0]}, Cols: {out.shape[1]}")
if changed is not None:
    print(f"Changed-cell ratio vs file A (on common pred cols): {changed*100:.2f}%")


[DONE] Saved -> JW_Tweedie_median.csv
Rows: 70, Cols: 168
Changed-cell ratio vs file A (on common pred cols): 20.65%


/var/folders/4n/dwz2ms357klcx0c9vw43n2b40000gn/T/ipykernel_8692/848183401.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[pred_cols] = ens
/var/folders/4n/dwz2ms357klcx0c9vw43n2b40000gn/T/ipykernel_8692/848183401.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[pred_cols] = ens
/var/folders/4n/dwz2ms357klcx0c9vw43n2b40000gn/T/ipykernel_8692/848183401.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider j